# Run xpander.ai Agent with Custom Handler

This notebook demonstrates how to initialize, wrap, and interact with an xpander.ai agent using a custom Python handler (`MyAgent`).
It includes loading credentials, listing tools, starting a chat, and printing message history.

In [1]:
import json
from tabulate import tabulate
from xpander_sdk import XpanderClient, Agent
from my_agent import MyAgent
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'tabulate'

## Load Environment & Agent Config
Loads credentials from `.env` and agent metadata from `xpander_config.json`.

In [ ]:
load_dotenv()

with open('xpander_config.json', 'r') as config_file:
    xpander_config: dict = json.load(config_file)

## Initialize Agent Instance
Connects to xpander.ai BaaS (Backend-as-a-service) and loads the agent specified in the config.

In [ ]:
xpander_client = XpanderClient(api_key=xpander_config.get("api_key"))
xpander_agent: Agent = xpander_client.agents.get(agent_id=xpander_config.get("agent_id"))

In [ ]:
print(f"Loaded Agent: {xpander_agent.name}")

## List Agent Tools
Displays all tools the agent can use, with descriptions and parameter details.

In [ ]:
tools = xpander_agent.get_tools()
tools_table = []

for tool in tools:
    func = tool.get("function", {})
    name = func.get("name", "")
    description = func.get("description", "").strip().replace("\n", " ")[:200] + "..."
    params = func.get("parameters", {}).get("properties", {})
    formatted_params = json.dumps(params)
    tools_table.append([name, description, formatted_params])

print("Agent's attached tools:")
print(tabulate(tools_table, headers=["Name", "Description", "Params JSON"], tablefmt="grid"))

## Wrap Agent with Custom Handler
Creates a custom agent interface (`MyAgent`) that controls how tasks are handled.

In [ ]:
agent = MyAgent(xpander_agent)

## Start a Chat with the Agent
Sends a first message (`Hi!`) and runs the custom handler loop.

In [ ]:
thread = await agent.chat("Hi!")

## View Execution Result (First Message)
After sending the first message, we'll fetch the agent's execution result. This includes:
- `status`: Current state of the task (`PENDING`, `EXECUTING`, `PAUSED`, `ERROR`, or `COMPLETED`)
- `result`: The agent's final answer or error message

In [ ]:
execution_result = xpander_agent.retrieve_execution_result()
print("status: ", execution_result.status.value)
print("result: ", execution_result.result)

## View Agent Message History
Displays messages from the current conversation thread in a readable table.

In [ ]:
print("The agent's thread:")
messages = xpander_agent.messages

table = []
for msg in messages:
    role = msg.get("role", "")
    content = msg.get("content", "")
    content_preview = content.strip().replace("\n", " ")
    content_preview = (content_preview[:100] + "...") if len(content_preview) > 100 else content_preview
    tool_calls = msg.get("tool_calls", "")
    tool_calls_json = json.dumps(tool_calls) if tool_calls else ""
    table.append([role, content_preview, tool_calls_json])

print(tabulate(table, headers=["Role", "Content", "Tool Calls"], tablefmt="grid"))

## Continue the Chat
Sends a follow-up message in the same thread to preserve context.

In [ ]:
await agent.chat("What can you do ?", thread)

## View Execution Result (Follow-Up)

This displays the execution result after the second chat message.  
Use it to verify if the agent completed the task or encountered an issue.

In [ ]:
execution_result = xpander_agent.retrieve_execution_result()
print("status: ", execution_result.status.value)
print("result: ", execution_result.result)